In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

#impore pipeline libraries
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [4]:
df_new_date = pd.read_csv("data/preprocessed_train_data_with_date_train.csv")
df_new_holi = pd.read_csv("data/data_with_holidays_and_lengths.csv")

In [5]:
df_new_date.head()

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,arr_time_of_day,route,departure_date,arrival_date,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,morning,TUN → IST,2016-01-16,2016-01-16,9.4,9.9,23.6,12.5,0.0,16.5
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,evening,DJE → NTE,2016-01-17,2016-01-17,11.7,0.0,41.2,2.7,2.6,8.8
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,night,TUN → MED,2016-01-20,2016-01-21,11.1,1.3,6.8,22.3,23.9,11.8
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,night,IST → TUN,2016-01-21,2016-01-21,5.2,0.0,10.8,10.3,0.0,5.0
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,evening,CMN → TUN,2016-01-22,2016-01-22,16.0,0.0,9.8,10.6,2.0,4.6


In [7]:
df_new_date.isnull().sum()

departure_point    0
arrival_point      0
departure_time     0
arrival_time       0
flight_status      0
aircraft_code      0
target             0
duration           0
dep_hour           0
dep_day            0
dep_month          0
dep_dayofweek      0
dep_quarter        0
dep_season         0
dep_is_weekend     0
dep_time_of_day    0
arr_hour           0
arr_day            0
arr_month          0
arr_dayofweek      0
arr_quarter        0
arr_season         0
arr_is_weekend     0
arr_time_of_day    0
route              0
departure_date     0
arrival_date       0
dep_temp           0
dep_precip         0
dep_wind           0
arr_temp           0
arr_precip         0
arr_wind           0
dtype: int64

In [8]:
df_new_holi.head()

,Unnamed: 0,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,Country,City,departure_date,is_holiday,holiday_length
0,0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-60.0,9300.0,Tunisia,Tunis,2016-01-16,0,0
1,1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-21.0,10200.0,Tunisia,Djerba,2016-01-17,0,0
2,2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-4.0,15600.0,Tunisia,Tunis,2016-01-20,0,0
3,3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-142.0,10200.0,Turkey,Istanbul,2016-01-21,0,0
4,4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-40.0,8700.0,Morocco,Casablanca,2016-01-22,0,0


In [9]:
df_new_date = pd.concat([
    df_new_date.reset_index(drop=True),
    df_new_holi[['Country', 'City', 'is_holiday', 'holiday_length']].reset_index(drop=True)
], axis=1)


In [13]:
df_new_date

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,9.4,9.9,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,11.7,0.0,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,11.1,1.3,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,5.2,0.0,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,16.0,0.0,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,16.8,0.0,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,9.6,1.2,10.7,17.3,0.0,12.5,France,Paris,0,0
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,19.1,0.0,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,10.2,1.2,8.2,16.7,0.0,19.6,France,Paris,0,0


In [11]:
df_new_date.isnull().sum()

departure_point    0
arrival_point      0
departure_time     0
arrival_time       0
flight_status      0
aircraft_code      0
target             0
duration           0
dep_hour           0
dep_day            0
dep_month          0
dep_dayofweek      0
dep_quarter        0
dep_season         0
dep_is_weekend     0
dep_time_of_day    0
arr_hour           0
arr_day            0
arr_month          0
arr_dayofweek      0
arr_quarter        0
arr_season         0
arr_is_weekend     0
arr_time_of_day    0
route              0
departure_date     0
arrival_date       0
dep_temp           0
dep_precip         0
dep_wind           0
arr_temp           0
arr_precip         0
arr_wind           0
Country            0
City               0
is_holiday         0
holiday_length     0
dtype: int64

In [12]:
df_new_date.to_csv("data/preprocessed_train_data_with_date_hol_concat.csv", index=False)